In [1]:
import json

import pandas as pd
import numpy as np

import networkx as nx
from graph_encoders import *

from linear_algebra_utils import *

from sklearn.linear_model import LinearRegression

In [2]:
X_train = pd.read_csv('../data/crop_mapping/node_reduced_train.csv')
X_test = pd.read_csv('../data/crop_mapping/node_reduced_test.csv')

with open('../data/crop_mapping/reduced_nodes.json', 'r') as file:
    nodes = json.load(file)

In [4]:
node_labels = list(nodes.keys())

In [5]:
# This is a placeholder graph to replace the graph structure obtained from a causal discovery algorithm.
# It does not describe any heuristics for expert knowledge in polarimetry/optics.
Q = nx.DiGraph()
Q.add_nodes_from(node_labels)
Q.add_edges_from([
    ('sig','R'), ('R','PH'), ('PH','rvi'),
    ('paul','RGB'), ('paul','krog'), ('RGB','yam'), ('krog', 'yam'),
    ('HA','free'), ('L','free')
])

In [6]:
dimensions = {k: len(nodes[k]) for k in node_labels}
bases = construct_standard_bases(dimensions)

nx.set_node_attributes(Q,dimensions,'dimension')
nx.set_node_attributes(Q,bases,'basis')

In [7]:
# Determine representation maps by fitting linear maps at each edge of the quiver
rep_maps={}
for e in list(Q.edges):
    rep_map = []
    X_1 = X_train[nodes[e[0]]]
    X_2 = X_train[nodes[e[1]]]

    for i in range(len(list(X_2.columns))):
        y = X_2.iloc[:,i]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_1,y)
        rep_map.append(model.coef_)
    
    rep_map = np.array(rep_map)
    rep_maps[e] = rep_map
nx.set_edge_attributes(Q,rep_maps,'rep map')

In [8]:
with open('../data/crop_mapping/quiver_representation.json', 'w') as file:
    json.dump(
        Q, file,
        cls=NetworkXQuiverRepresentationEncoder
    )